In [1]:
from novartis_config import questions, optional_info, additional_qs, req_sub_dict
import sqlite3, random, re, copy, pickle
from utils import sub_reqs, get_entities
from synthetic_pharma_data import get_settings
from random import shuffle

In [2]:
import os
import json

In [3]:
download_path = '../deepPavlov/DeepPavlov/download/pharma'

In [4]:
from collections import OrderedDict

In [5]:
tag_dict = {1: 'what_question', 2: 'should_question', 3: 'usage_question', 
            4: 'dose_question', 5: 'side_effect_question', 6: 'store_question', 7: 'ingredient_question'}

In [6]:
question_tag_dict = OrderedDict(tag_dict)

In [7]:
question_tag_dict

OrderedDict([(1, 'what_question'),
             (2, 'should_question'),
             (3, 'usage_question'),
             (4, 'dose_question'),
             (5, 'side_effect_question'),
             (6, 'store_question'),
             (7, 'ingredient_question')])

In [8]:
database = 'NovartisQA'
output = 'dialogues_new_'

In [9]:
entity_dict = get_entities('entities.txt')
context_keys = sorted(entity_dict.keys())

In [9]:
# connect to database
conn = sqlite3.connect(database+'.sqlite')
cur = conn.cursor()

# get list of id for questions from table
cur.execute('SELECT id FROM Questions')
question_ids = cur.fetchall()

In [10]:
openings = ['hello', 'hi', 'good morning', 'hi there', 'good evening', 'good afternoon']

In [11]:
endings = ['thank you','thanks','ok thanks']

In [12]:
def gen_opening(i, greeting):
    user = {}
    user['index'] = i
    user['text'] = greeting
    user['goal'] = {'greeting': greeting}
    user['db_result'] = None
    user['dialog_acts'] = []
    bot = {"index": i, "text": "hello, what can I help you with today?", 
           "dialog_acts": [{"act": "welcomemsg", "slot": []}]}
    return user, bot

In [13]:
def gen_ending(i, bye):
    user = {}
    user['index'] = i
    user['text'] = bye
    user['goal'] = {'bye': bye}
    user['db_result'] = None
    user['dialog_acts'] = []
    bot = {"index": i, "text": 'no problem', "dialog_acts": [{"act": 'bye', "slot": []}]}
    return user, bot

In [14]:
question_ids

[(1,), (2,), (3,), (4,), (5,), (6,), (7,)]

In [15]:
def select_question(question_id, id2tag):
    cur.execute('SELECT * FROM Questions WHERE id = ?', (question_id, ))
    question_data = cur.fetchall()[0]
    tag = id2tag[question_id]
    question_formats = [q for q in list(question_data[1:5]) if q]
    # sql query
    query = question_data[5]
    # answer
    answer = question_data[6]
    # required slots
    required_fields = question_data[7].split(',')
    question_text = question_formats[random.randint(0, len(question_formats)-1)] + '?'
    return question_text, tag, query, answer, required_fields

In [16]:
select_question(2, question_tag_dict)

('can i take <drug_name>?',
 'should_question',
 "SELECT advice \n                         FROM Contraindications JOIN Drugs JOIN Conditions \n                         ON drug_id = Drugs.id AND condition_id = Conditions.id\n                         WHERE Drugs.name = '<drug_name>' AND Conditions.condition = '<condition>'",
 'answer_text = cur.fetchall()[0][0]',
 ['<drug_name>', '<condition>'])

In [17]:
# {"index": i, "text": question_text, "goals" : {"question": tag}, "db_result": null, "dialog_acts": [{"slots": [[]], "act": "question"}]}
def gen_question(i, question_text, tag, slots):
    goal = {"question": tag}
    dialog_acts = []
    dialog_acts["act"] = "question"
    dialog_acts["slots"] = slots
    return {"index": i, "text": question_text, "goal": goal, "db_result": None, "dialog_acts": dialog_acts}

In [30]:
def answer_quest(curr_dialogue, query, context_details):
    query = sub_reqs(cur, query, req_sub_dict, context_details)
    print(query)
    cur.execute(query)
    try:
        answer = cur.fetchall()[0][0]
        print(answer)
        curr_dialogue += [str(-1) + ' ' + answer]
    except Exception as e:
        print(e)
        curr_dialogue += [str( - 1) + ' '
                       + 'sorry, there is no information for that']
    return curr_dialogue

In [40]:
context_details = get_settings()

In [41]:
context_details

{'<age_group>': 'elderly',
 '<condition>': 'heart_problems',
 '<delivery>': 'subcutaneous',
 '<drug_name>': 'Nifedipine',
 '<strength>': '50',
 '<symptom>': 'fever',
 '<units>': 'mg'}

In [42]:
context_details['<drug_name>'] = 'HYDROCORTISONE'
context_details['<condition>'] = 'anxiety'

In [43]:
context_details

{'<age_group>': 'elderly',
 '<condition>': 'anxiety',
 '<delivery>': 'subcutaneous',
 '<drug_name>': 'HYDROCORTISONE',
 '<strength>': '50',
 '<symptom>': 'fever',
 '<units>': 'mg'}

In [49]:
_, _, query, answer, _= select_question(1, question_tag_dict)

In [50]:
query

"SELECT indication FROM Drugs WHERE name = '<drug_name>'"

In [51]:
answer

'answer_text = cur.fetchall()[0][0]'

In [52]:
query = sub_reqs(cur, query, req_sub_dict, context_details)

In [53]:
query

"SELECT indication FROM Drugs WHERE name = 'HYDROCORTISONE'"

In [54]:
cur.execute(query)

In [55]:
cur.fetchall()

[('Hydrocortisone acetate suppositories are indicated for use in inflamed hemorrhoids, post-irradiation (factitial) proctitis; as an adjunct in the treatment of chronic ulcerative colitis; cryptitis; and other inflammatory conditions of anorectum and pruritus ani.',)]

In [ ]:
exec(globals()['answer'], globals())

In [31]:
answer_quest([], query, context_details)

SELECT advice 
                         FROM Contraindications JOIN Drugs JOIN Conditions 
                         ON drug_id = Drugs.id AND condition_id = Conditions.id
                         WHERE Drugs.name = 'HYDROCORTISONE' AND Conditions.condition = 'anxiety'
list index out of range


['-1 sorry, there is no information for that']

In [61]:
class gen_dialogue(object):
    """
    generate dialogues for a question, given question_id
    """
    def __init__(self, context_details, question_id, id2tag):
        self.context = context_details
        self.question_text, self.tag, self.query, self.answer, self.required_fields = select_question(question_id, 
                                                                                                      id2tag)
        self.curr_slots = {}
        # randomly add additional information for certain questions
        for opt, versions in optional_info.items():
            if opt in self.required_fields and random.randint(0, 1):
                version = random.choice(versions)
                question_text = self.question_text[:-1] + ' ' + version + '?'
        
    def update(self, field):
        if field not in self.required_fields:
            self.curr_slots.pop(field, None)
            self.required_fields.append(field)
        self.context[field] = get_settings()[field]
        
    def sub_reqs(self, text, req_sub_dict, act=None):
        # for user's question, inform, update
        context_details = self.context
        dialog_acts = {}
        dialog_acts['slots'] = []
        goals ={}
        for requirement in req_sub_dict.keys():
            if requirement in text:
                exec(req_sub_dict[requirement][0], locals())
                # test = cur.fetchall()
                exec(req_sub_dict[requirement][1], locals())
                value = locals()['sub_data']
                text = re.sub(requirement, value, text)
                self.curr_slots[requirement] = value
                self.required_fields.remove(requirement)
                dialog_acts['slots'].append([requirement, value])
        if act:
            dialog_acts['act'] = act
            if act == 'question':
                goals['question'] = self.tag
            else:
                goals['act'] = act
        return text, goals, [dialog_acts]
    
    def answer_question(self, req_sub_dict):
        # bot answer the question
        assert not self.required_fields
        query = sub_reqs(cur, self.query, req_sub_dict, self.context)
        cur.execute(query)
        try:
            exec(self.answer, globals())
            answer_text = globals()['answer_text']
        except:
            answer_text = 'sorry, there is no information for that.'
            
        dialog_acts = {'act' : 'information_retrieval'}
        return answer_text, [dialog_acts]
    
    def request_information(self, requirement, additional_qs):
        assert requirement in self.required_fields
        text = additional_qs[requirement]
        for field, value in self.context.items():
            if field in text:
                text = re.sub(field, value, text)
        dialog_acts = {'act' : 'request_information'}
        return text, [dialog_acts]

In [62]:
# generate synthetic dialogue data
def gen_data(num_dialogues):
    dialogues = []
    for d in range(num_dialogues):
        curr_dialogue = []
        curr_idx = 0
    
        # maybe say hello
        if random.randint(0, 1) == 1:
            user_line, bot_line = gen_opening(curr_idx, random.choice(openings))
            curr_dialogue.append(user_line)
            curr_dialogue.append(bot_line)
            curr_idx += 1

        # select a question and format
        question_id = random.randint(1, len(question_ids))
    
        # generate required fields from available options
        context_details = get_settings()
        dialogue = gen_dialogue(context_details, question_id, question_tag_dict)
        required_fields = dialogue.required_fields[:]
    
        # ask the question
        text, goals, dialog_acts = dialogue.sub_reqs(dialogue.question_text, req_sub_dict, act='question')
        user_line = {'index': curr_idx, 'text': text, 'goals': goals, "dialog_acts": dialog_acts}
        curr_dialogue.append(user_line)
    
        # TODO not picking up conditions, check for required information
        for field in dialogue.required_fields[:]:
            text, dialog_acts = dialogue.request_information(field, additional_qs)
            bot_line = {'index': curr_idx, 'text': text, 'dialog_acts': dialog_acts}
            curr_dialogue.append(bot_line)
            curr_idx += 1
            text, goals, dialog_acts = dialogue.sub_reqs(field, req_sub_dict, 'inform')
            user_line = {'index': curr_idx, 'text': text, 'goals': goals, "dialog_acts": dialog_acts}
            curr_dialogue.append(user_line)
        
        text, dialog_acts = dialogue.answer_question(req_sub_dict)
        bot_line = {'index': curr_idx, 'text': text, 'dialog_acts': dialog_acts}
        curr_dialogue.append(bot_line)
        curr_idx += 1
    
        # correct information
        num_updates = random.randint(0, min(2,len(required_fields)))
        requirements = random.sample(required_fields, num_updates)
        for requirement in requirements:
            dialogue.update(requirement)
            text, goals, dialog_acts = dialogue.sub_reqs(requirement, req_sub_dict, 'update')
            user_line = {'index': curr_idx, 'text': 'actually i meant '+ text, 'goals': goals, 'dialog_acts': dialog_acts}
            curr_dialogue.append(user_line)
            bot_line = {'index': curr_idx, 'text': 'ok, is there anything else to update?', 'dialog_acts': [{'act':'check_update'}]}
            curr_dialogue.append(bot_line)
            curr_idx += 1
        if num_updates > 0:
            user_line = {'index': curr_idx, 'text': 'no', 'goals': {}, 'dialog_acts': []}
            curr_dialogue.append(user_line)
            text, dialog_acts = dialogue.answer_question(req_sub_dict)
            bot_line = {'index': curr_idx, 'text': text, 'dialog_acts': dialog_acts}
            curr_dialogue.append(bot_line)
            curr_idx += 1

        # maybe say thank you
        if random.randint(0,1) == 1:
            user_line, bot_line = gen_ending(curr_idx, random.choice(endings))
            curr_dialogue.append(user_line)
            curr_dialogue.append(bot_line)
            curr_idx += 1

        dialogues.append(curr_dialogue)
    return dialogues

In [63]:
gen_data(2)

[[{'db_result': None,
   'dialog_acts': [],
   'goal': {'greeting': 'good afternoon'},
   'index': 0,
   'text': 'good afternoon'},
  {'dialog_acts': [{'act': 'welcomemsg', 'slot': []}],
   'index': 0,
   'text': 'hello, what can I help you with today?'},
  {'dialog_acts': [{'act': 'question',
     'slots': [['<drug_name>', 'Anthim'], ['<symptom>', 'depression']]}],
   'goals': {'question': 'side_effect_question'},
   'index': 1,
   'text': 'is it normal to have depression with Anthim?'},
  {'dialog_acts': [{'act': 'information_retrieval'}],
   'index': 1,
   'text': 'depression is a common side effect of Anthim. The following clinically important adverse reactions are described elsewhere in the labeling: Hypersensitivity and Anaphylaxis [see Warnings and Precautions ( 5.1 )].'},
  {'dialog_acts': [{'act': 'update', 'slots': [['<symptom>', 'bleeding']]}],
   'goals': {'act': 'update'},
   'index': 2,
   'text': 'actually i meant bleeding'},
  {'dialog_acts': [{'act': 'check_update'}],


In [64]:
gen_data(2)

[[{'db_result': None,
   'dialog_acts': [],
   'goal': {'greeting': 'good evening'},
   'index': 0,
   'text': 'good evening'},
  {'dialog_acts': [{'act': 'welcomemsg', 'slot': []}],
   'index': 0,
   'text': 'hello, what can I help you with today?'},
  {'dialog_acts': [{'act': 'question', 'slots': [['<drug_name>', 'Oracea']]}],
   'goals': {'question': 'ingredient_question'},
   'index': 1,
   'text': 'does Oracea contain <ingredient>?'},
  {'dialog_acts': [{'act': 'request_information'}],
   'index': 1,
   'text': 'what version of Oracea is that, for example: oral, intravenous or subcutaneous?'},
  {'dialog_acts': [{'act': 'inform',
     'slots': [['<delivery>', 'subcutaneous']]}],
   'goals': {'act': 'inform'},
   'index': 2,
   'text': 'subcutaneous'},
  {'dialog_acts': [{'act': 'information_retrieval'}],
   'index': 2,
   'text': 'DOXYCYCLINE,HYPROMELLOSE,FERRIC OXIDE RED,FERRIC OXIDE YELLOW,METHACRYLIC ACID - ETHYL ACRYLATE COPOLYMER (1:1) TYPE A,POLYETHYLENE GLYCOL,POLYSORBATE 8